In [1]:
'''
Algoritmo Recocido Simulado
De manera conceptual
1. Estado inicial = T
2. T = caliente
3. Repita hasta que T= frío
4. Perturbe el sistema (genere un estado al azar con distancia proporcional a T)
5. Calcule la energía del sistema
6. Si ΔE es < 0
 Entonces acepte el nuevo estado
 Sino acepte el nuevo estado con probabilidad = exp(-ΔE/KT}
7. Si ΔE < 0 decreció en las últimas x temperaturas
 Entonces T=0.9*T
 Sino T=frío
7. Retorne con el sistema en el último estado

Y llevado a código:
i := iinic ; i es la solución actual
f(i) := f eval inicial ; f es la función de evaluación a minimizar
Repite
For T:= Tinic to 1 do; Tinic es la temperatura Inicial del sistema
{Generar j ϵ Sj; (S es el espacio de soluciones)
Si f(j) < f(i)
entonces i := j
sino, si exp[ f(i)-f(j)/C.T] > random [0,1]
entonces i := j
T:= T-1
Si f(i)= f(iT+n) entonces return f(i)
}
'''
import matplotlib as plt
import random
import pickle
import os
import json
import logging
from datetime import datetime, timedelta

'''
Voy a modelar el algoritmo para que al principio tome como primer camino válido el tomado desde una ciudad en particular (elegir una al azar) y tomar el mínimo costo de llegar hacia otra ciudad (el minimo de todos los posibles desde dicha ciudad) y sige iterando hasta llegar a la última ciudad y sumarle el costo de llegar desde la última hasta la primera
Para perturbar al sistema voy a hacer un shuffle de las ciudades dentro del vector camino_actual dejando fijas una cierta cantidad de ciudades en base a su costo (los N menores recorridos quedaran fijos luego del shuffle, este valor N depende directamente de T, cuanto mas chico T más chico N hasta un máximo de cantCiudades-1)

Para las pruebas se tomarán los problemas con Geometria euclideana en 2D: att48, kroB100, kroB150, kroB200, rat575.
Opcionalmente se realizarán pruebas con eil51, berlin52, st70, eil76, eil101, tsp225, att532 y rat783.
'''

def _leerArchConf(archivo):
    arch = open(archivo)
    # Las primeras 3 lineas del archivo se obvian
    for a in range(3):
        arch.readline()
    # Se lee la dimension del problema
    cantCiu = int(arch.readline().split("DIMENSION: ")[1])
    # Las siguientes 2 lineas del archivo se obvian
    for a in range(2):
        arch.readline()

    vec_ciudades = []
    vec_ciudades_map = []

    line = arch.readline()
    aux = [a for a in line.split(' ') if a is not '' and a is not '\n']
    while (line != 'EOF\n'):
        vec_ciudades.append([int(a) for a in aux])
        line = arch.readline()
        aux = [a for a in line.split(' ') if a is not '' and a is not '\n']

    # Se arman las tuplas de ciudades [(nro1, [x1, y1]),(nro2, [x2, y2]),...]
    vec_ciudades_map.append(map(lambda e: (e[0], e[1:]), vec_ciudades))

    # Se arma la lista de las tuplas de ciudades
    vec_ciudades_list = [list(a) for a in vec_ciudades_map][0]
    arch.close()

    vec_costCiu = []
    aux = [a for a in vec_ciudades_list]
    while (len(aux) > 0):
        ciudad_actual = aux.pop(0)
        for b in aux:
            costo_x = (ciudad_actual[1][0] - b[1][0])**2
            costo_y = (ciudad_actual[1][1] - b[1][1])**2
            costo = (costo_x+costo_y)**(1/2)
            vec_costCiu.append(costo)


    vecmat_costCiu = []

    vec_costCiu_aux = [a for a in vec_costCiu]
    for a in range(cantCiu):
        
        #TODO: optimizar -> c no se usa y se puede hacer mejor un append con ceros
        # Se inicializa en ceros el vector de costos de las ciudades que se utilizara para llenar la matriz de costos
        for c in range(a+1):
            vecmat_costCiu.append(0)
        
        # se llena el triangulo superior de la matriz
        for b in range(cantCiu-a-1):
            vecmat_costCiu.append(vec_costCiu_aux[0])
            vec_costCiu_aux = vec_costCiu_aux[1:]

    # se traspone el triangulo superior y se almacenan sus valores
    for a in range(cantCiu):
        for b in range(cantCiu):
            if b < a:
                # si la columna es menor a la fila, se copia el valor de su opuesto
                vecmat_costCiu[cantCiu*a+b] = vecmat_costCiu[cantCiu*b+a]

    # se termina de componer la matriz
    mat_costCiu = []
    for a in range(cantCiu):
        mat_costCiu.append(vecmat_costCiu[:cantCiu])
        vecmat_costCiu = vecmat_costCiu[cantCiu:]

    # se compone el objeto (dict) para agilizar el calculo de costos (costo de N a M).
    obj_costCiu = dict()
    for a in range(1,cantCiu+1):
        indice_1 = "ciudad{}".format(a)
        obj_costCiu[indice_1] = dict()
        for b in range(1,cantCiu+1):
            if (a is not b):
                indice_2 = "ciudad{}".format(b)
                obj_costCiu[indice_1][indice_2] = mat_costCiu[a-1][b-1]

    return cantCiu, vec_costCiu, mat_costCiu, obj_costCiu


def generarPrimerCamino(mat_costCiu, cantCiu, n_ciudad_inicio):
    aux = []
    for a in mat_costCiu:
        vec = []
        for b in a:
            vec.append(b)
        aux.append(vec)

    recorrido = []
    ciudad_actual = n_ciudad_inicio
    recorrido.append(ciudad_actual)
    while (len(recorrido) < cantCiu):
        for a in recorrido:
            aux[ciudad_actual-1][a-1] = -1
        ciudad_actual = 1 + aux[ciudad_actual-1].index(min([a for a in aux[ciudad_actual-1] if (a is not -1 and a is not 0)]))
        recorrido.append(ciudad_actual)
    return recorrido

def calcularCostoRecorrido(mat_costCiu, recorrido_actual):
    aux = [a for a in mat_costCiu]
    costo_rec_actual = 0
    costo = 0
    costos = []
    for a in range(len(recorrido_actual)-1):
        costo = aux[recorrido_actual[a]-1][recorrido_actual[a+1]-1]
        costos.append(costo)
        costo_rec_actual += costo
    costo = aux[recorrido_actual[-1:][0]-1][recorrido_actual[0]-1]
    costos.append(costo)
    costo_rec_actual += costo
    return costo_rec_actual, costos

def generarPerturbacion(mat_costCiu, cantCiu, recorrido_actual, t, T, factor_de_aprendizaje):
    # Se reformula la lista de recorridos para tener la informacion necesaria en la toma de decision
    rec_actual_redundante = []
    for a in range(0,len(recorrido_actual)-1):
        rec_actual_redundante.append((costos_actual[a],[recorrido_actual[a],recorrido_actual[a+1]]))
    rec_actual_redundante.append((costos_actual[a],[recorrido_actual[a+1],recorrido_actual[0]]))

    # Se obtienen la cantidad de recorridos a mezclar
    if (t/T > factor_de_aprendizaje):
        N = cantCiu #ayudo capacidad de exploración en las 2/3 partes del proceso
    else:
        t = t/factor_de_aprendizaje
        N = int((cantCiu*t)/T)-1

    # Se obtienen los cantCiu-N menores recorridos a persistir y se convierte a ciudades a inmovilizar en la proxima solucion
    aux = [a for a in rec_actual_redundante]
    # TODO: ver si el comentar el Sort facilita capacidad de exploracion
    aux.sort(reverse=False)

    ciudades_a_inmovilizar = []
    for a in [a[1] for a in aux[:cantCiu-N]]:
        for b in a:
            ciudades_a_inmovilizar.append(b)
    # Se filtran las ciudades duplicadas
    ciudades_a_inmovilizar = list(set(ciudades_a_inmovilizar))
    # una opcion para mantener el orden de las ciudades durante el filtrado:
    #aux_set = set()
    #ciudades_a_inmovilizar = [x for x in ciudades_a_inmovilizar if x not in used and (used.add(x) or True)]

    ciudades_a_mezclar = []
    for a in [a[1] for a in aux[cantCiu-N:]]:
        for b in a:
            if (b not in ciudades_a_inmovilizar):
                ciudades_a_mezclar.append(b)
    # Se filtran las ciudades duplicadas
    ciudades_a_mezclar = list(set(ciudades_a_mezclar))

    # Se genera un recorrido posible
    recorrido_posible = []
    for a in recorrido_actual:
        if (a in ciudades_a_inmovilizar):
            recorrido_posible.append(a)
        else:
            ciudad = ciudades_a_mezclar.pop(random.randint(0,len(ciudades_a_mezclar)-1))
            recorrido_posible.append(ciudad)
    return recorrido_posible


# Se especifica la semilla para generar numeros pseudoaleatorios
random.seed(26)

# Parametros del algoritmo
T = 2000
C = 20
factor_de_aprendizaje = 0.3

# Se lee el archivo de configuración
cantCiu, vec_costCiu, mat_costCiu, obj_costCiu = _leerArchConf("ALL_tsp/att_euc2d/att48.tsp")

# Se genera una primer solución resultante de partir de una ciudad tomada de manera aleatoria y recorriendo la ciudad de menor costo.
recorrido_actual = generarPrimerCamino(mat_costCiu, cantCiu, random.randint(1,cantCiu))

# Se obtiene el costo del recorrido actual
costo_rec_actual, costos_actual = calcularCostoRecorrido(mat_costCiu, recorrido_actual)

for t in range(T, 1, -1):
    # Se genera una solución a partir de la actual
    recorrido_posible = generarPerturbacion(mat_costCiu, cantCiu, recorrido_actual, t, T, factor_de_aprendizaje)
    costo_rec_posible, costos_posible = calcularCostoRecorrido(mat_costCiu, recorrido_posible)

    aceptar_nuevo_estado = False
    if costo_rec_posible < costo_rec_actual:
        aceptar_nuevo_estado = True
    # sino, si exp[ f(i)-f(j)/C.T] > random [0,1]        
    elif (2.71**((costo_rec_actual-costo_rec_posible)/20*t) > random.uniform(0,1)):
        aceptar_nuevo_estado = True
        
    if (aceptar_nuevo_estado):
        recorrido_actual, costo_rec_actual, costos_actual = recorrido_posible, costo_rec_posible, costos_posible
        




In [47]:
'''
Las siguientes celdas del Notebook se mantienen porque el algoritmo está en desarrollo....
'''


archivo = "ALL_tsp/att_euc2d/att48.tsp"
arch = open(archivo)
# Las primeras 3 lineas del archivo se obvian
for a in range(3):
    arch.readline()
# Se lee la dimension del problema
cantCiu = int(arch.readline().split("DIMENSION: ")[1])
# Las siguientes 2 lineas del archivo se obvian
for a in range(2):
    arch.readline()

vec_ciudades = []
vec_ciudades_map = []

line = arch.readline()
aux = [a for a in line.split(' ') if a is not '' and a is not '\n']
while (line != 'EOF\n'):
    vec_ciudades.append([int(a) for a in aux])
    line = arch.readline()
    aux = [a for a in line.split(' ') if a is not '' and a is not '\n']

# Se arman las tuplas de ciudades [(nro1, [x1, y1]),(nro2, [x2, y2]),...]
vec_ciudades_map.append(map(lambda e: (e[0], e[1:]), vec_ciudades))

# Se arma la lista de las tuplas de ciudades
vec_ciudades_list = [list(a) for a in vec_ciudades_map][0]
arch.close()

vec_costCiu = []
aux = [a for a in vec_ciudades_list]
ciudad_actual = aux.pop(0)
for b in aux:
    costo_x = (ciudad_actual[1][0] - b[1][0])**2
    costo_y = (ciudad_actual[1][1] - b[1][1])**2
    costo = (costo_x+costo_y)**(1/2)
    vec_costCiu.append(costo)

print(vec_costCiu)



'''
Se calculan los costos entre las ciudades (resultando como en A*)
vec_costCiu = []
aux = [a for a in vec_ciudades_list]
while (len(aux) > 0):
    ciudad_actual = aux.pop(0)
    for b in aux:
        costo_x = (ciudad_actual[1][0] - b[1][0])**2
        costo_y = (ciudad_actual[1][1] - b[1][1])**2
        costo = (costo_x+costo_y)**(1/2)
        vec_costCiu.append(costo)
'''
a = 2
# asd

[4726.653149957166, 1204.3492018513568, 6362.502102160753, 3656.991249647721, 3129.520889848796, 2413.5223222502004, 562.3041881401916, 462.0822437618654, 5653.503338638795, 1712.356563336036, 1603.5663378856516, 2367.1977103740196, 2200.3654241966265, 1289.4452295464123, 1003.8849535678877, 3832.687046968484, 2257.5989457828864, 3418.9039179245738, 2266.984781598677, 2956.020466776237, 719.9569431570196, 1699.3133907552192, 5278.156591083671, 2577.884791840008, 6075.466237252907, 3464.6784843618607, 2653.682912482198, 3624.6459137410925, 3114.9210262862202, 1573.1576526209953, 3950.066455137179, 1747.3591502607585, 2141.2381931957034, 6754.234967781325, 2382.115026609756, 3305.89594512592, 1028.9047574970193, 3529.2883135272473, 824.3136538963794, 2187.0002286236736, 4819.396227744716, 3488.9872456058074, 1946.9003569777267, 6834.318181062395, 1541.7292239560097, 2378.534422706554, 3743.0792136955906]


In [151]:
mat_costCiu = [
    [ 0, 3, 6, 9,12, 3, 6, 9,12],
    [ 3, 0, 6, 7, 8, 1, 2, 2, 3],
    [ 6, 6, 0, 3,10, 8, 5, 1, 2],
    [ 9, 7, 3, 0, 5,19,21, 5, 1],
    [12, 8,10, 5, 0, 8,25, 1,45],
    [ 3, 1, 8,19, 8, 0, 9, 8, 7],
    [ 6, 2, 5,21,25, 9, 0, 1, 1],
    [ 9, 2, 1, 5, 1, 8, 1, 0, 4],
    [12, 3, 2, 1,45, 7, 1, 4, 0]
    ]

cantCiu = 9
n_ciudad_inicio = 2
aux = [a for a in mat_costCiu]
recorrido = []
ciudad_actual = n_ciudad_inicio
recorrido.append(ciudad_actual)
print(recorrido)
print(aux[ciudad_actual-1])
while (len(recorrido) < cantCiu):
    for a in recorrido:
        aux[ciudad_actual-1][a-1] = -1

    ciudad_actual = 1 + aux[ciudad_actual-1].index(min([a for a in aux[ciudad_actual-1] if (a is not -1 and a is not 0)]))
    recorrido.append(ciudad_actual)
    print(recorrido)
    print(aux[ciudad_actual-1])

def calcularCostoRecorrido(mat_costCiu, recorrido_actual):
    costo_rec_actual = 0
    costo = 0
    costos = []
    for a in range(len(recorrido_actual)-1):
        costo = mat_costCiu[recorrido_actual[a]-1][recorrido_actual[a+1]-1]
        costos.append(costo)
        costo_rec_actual += costo
    costo = mat_costCiu[recorrido_actual[-1:][0]-1][recorrido_actual[0]-1]
    costos.append(costo)
    costo_rec_actual += costo
    return costo_rec_actual, costos

aux = [a for a in mat_costCiu]
print(calcularCostoRecorrido(aux,recorrido))

[2]
[3, 0, 6, 7, 8, 1, 2, 2, 3]
[2, 6]
[3, 1, 8, 19, 8, 0, 9, 8, 7]
[2, 6, 1]
[0, 3, 6, 9, 12, 3, 6, 9, 12]
[2, 6, 1, 3]
[6, 6, 0, 3, 10, 8, 5, 1, 2]
[2, 6, 1, 3, 8]
[9, 2, 1, 5, 1, 8, 1, 0, 4]
[2, 6, 1, 3, 8, 5]
[12, 8, 10, 5, 0, 8, 25, 1, 45]
[2, 6, 1, 3, 8, 5, 4]
[9, 7, 3, 0, 5, 19, 21, 5, 1]
[2, 6, 1, 3, 8, 5, 4, 9]
[12, 3, 2, 1, 45, 7, 1, 4, 0]
[2, 6, 1, 3, 8, 5, 4, 9, 7]
[6, 2, 5, 21, 25, 9, 0, 1, 1]
(21, [1, 3, 6, 1, 1, 5, 1, 1, 2])


In [160]:
mat_costCiu = [
    [ 0, 3, 6, 9,12, 3, 6, 9,12],
    [ 3, 0, 6, 7, 8, 1, 2, 2, 3],
    [ 6, 6, 0, 3,10, 8, 5, 1, 2],
    [ 9, 7, 3, 0, 5,19,21, 5, 1],
    [12, 8,10, 5, 0, 8,25, 1,45],
    [ 3, 1, 8,19, 8, 0, 9, 8, 7],
    [ 6, 2, 5,21,25, 9, 0, 1, 1],
    [ 9, 2, 1, 5, 1, 8, 1, 0, 4],
    [12, 3, 2, 1,45, 7, 1, 4, 0]
    ]
cantCiu = 9
recorrido_actual = [2, 6, 1, 3, 8, 5, 4, 9, 7]
costo_rec_actual = 21
costos_actual = [1, 3, 6, 1, 1, 5, 1, 1, 2]
t = 5
T = 20
factor_de_aprendizaje = 0.3

# Se reformula la lista de recorridos para tener la informacion necesaria en la toma de decision
rec_actual_redundante = []
for a in range(0,len(recorrido_actual)-1):
    rec_actual_redundante.append((costos_actual[a],[recorrido_actual[a],recorrido_actual[a+1]]))
rec_actual_redundante.append((costos_actual[a],[recorrido_actual[a+1],recorrido_actual[0]]))

# Se obtienen la cantidad de recorridos a mezclar
if (t/T > factor_de_aprendizaje):
    N = cantCiu #ayudo capacidad de exploración en las 2/3 partes del proceso
else:
    t = t/factor_de_aprendizaje
    N = int((cantCiu*t)/T)-1

# Se obtienen los cantCiu-N menores recorridos a persistir y se convierte a ciudades a inmovilizar en la proxima solucion
aux = [a for a in rec_actual_redundante]
# TODO: ver si el comentar el Sort facilita capacidad de exploracion
aux.sort(reverse=False)

ciudades_a_inmovilizar = []
for a in [a[1] for a in aux[:cantCiu-N]]:
    for b in a:
        ciudades_a_inmovilizar.append(b)
# Se filtran las ciudades duplicadas
ciudades_a_inmovilizar = list(set(ciudades_a_inmovilizar))
# una opcion para mantener el orden de las ciudades durante el filtrado:
#aux_set = set()
#ciudades_a_inmovilizar = [x for x in ciudades_a_inmovilizar if x not in used and (used.add(x) or True)]

ciudades_a_mezclar = []
for a in [a[1] for a in aux[cantCiu-N:]]:
    for b in a:
        if (b not in ciudades_a_inmovilizar):
            ciudades_a_mezclar.append(b)
# Se filtran las ciudades duplicadas
ciudades_a_mezclar = list(set(ciudades_a_mezclar))

# Se genera un recorrido posible
recorrido_posible = []
for a in recorrido_actual:
    if (a in ciudades_a_inmovilizar):
        recorrido_posible.append(a)
    else:
        ciudad = ciudades_a_mezclar.pop(random.randint(0,len(ciudades_a_mezclar)-1))
        recorrido_posible.append(ciudad)

'''
while (len(aux) > (cantCiu - N)):
    indice_anterior = aux.index(min(aux)))
    if (indice == len(aux)):
        # es el ultimo indice
        indice_posterior = 0

        recorridos_a_persistir.append(rec_actual_redundante[indice])
'''
# Se pasan los recorridos a persistir a ciudades a inmovilizar en la proxima solucion

a=4

In [179]:
C = 20
t = 1
costo_rec_actual=25
costo_rec_posible=28
(2.71**((costo_rec_actual-costo_rec_posible)/20*t) > random.uniform(0,1)) 

False

In [192]:
matriz = [
    [ 0, 3, 6, 9,12, 3, 6, 9,12],
    [ 3, 0, 6, 7, 8, 1, 2, 2, 3],
    [ 6, 6, 0, 3,10, 8, 5, 1, 2],
    [ 9, 7, 3, 0, 5,19,21, 5, 1],
    [12, 8,10, 5, 0, 8,25, 1,45],
    [ 3, 1, 8,19, 8, 0, 9, 8, 7],
    [ 6, 2, 5,21,25, 9, 0, 1, 1],
    [ 9, 2, 1, 5, 1, 8, 1, 0, 4],
    [12, 3, 2, 1,45, 7, 1, 4, 0]
    ]
cantCiu = 9
n_ciudad_inicio = 2
aux = []
for a in matriz:
    vec = []
    for b in a:
        vec.append(b)
    aux.append(vec)

recorrido = []
ciudad_actual = n_ciudad_inicio
recorrido.append(ciudad_actual)
while (len(recorrido) < cantCiu):
    for a in recorrido:
        aux[ciudad_actual-1][a-1] = -1
    ciudad_actual = 1 + aux[ciudad_actual-1].index(min([a for a in aux[ciudad_actual-1] if (a is not -1 and a is not 0)]))
    recorrido.append(ciudad_actual)
return recorrido

SyntaxError: 'return' outside function (<ipython-input-192-cb485c488406>, line 29)